<a href="https://colab.research.google.com/github/VanesaHM/ProyectoKaggle/blob/main/03_modelo_con_preprocesado_(KNN_%2B_OHE_%2B_RobustScaler)_y_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia**

Crear un modelo para predecir el rendimiento de los estudiantes en las pruebas Saber Pro

# **Descripción general**

Las Pruebas Saber Pro son exámenes estandarizados que se administran en Colombia para evaluar la calidad y el nivel de conocimiento y competencias de los estudiantes de educación superior, es decir, de instituciones de educación superior como universidades y tecnológicos. Estas pruebas son parte de los esfuerzos del Gobierno de Colombia para monitorear y mejorar la calidad de la educación superior en el país.

Estas Pruebas constan cinco componentes genéricos, Inglés, Lectura Crítica, Competencias Ciudadanas, Razonamiento Cuantitativo y Comunicación Escrita.

Tu tarea será crear un modelo de clasificación que para cada estudiante prediga qué desempeño va a tener: bajo, medio-bajo, medio-alto o alto.

# **Descripción**

El conjunto de datos contiene más de 50 columnas que describen de manera distintos aspectos de cada estudiante, incluyendo:

Información socieconómica: Describen características socieconómicas del estudiante como su estrato, educación de sus padres, estrato, entre otras.

Información de instituciones: Describen las instituciones de donde provienen los estudiantes.

Información del estudiante: Describe particularidades del estudiante como su edad, que programa estudian, la modalidad de estudio, etc.

Información estadística: Describe algunos coeficientes que equipos de estudio han desarrollado que podría ayudar a la clasificación.

Así como muchos otros datos que ayudan a clasificar de manera precisa los niveles de desempeño

# **Modelo con preprocesado  (KNN + OHE + RobustScaler) y XGBoost**



**1. Importar librerías**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, RobustScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score
)
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import pickle
import time


**2. Configuración visual**

In [ ]:
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="muted")

**3. Cargar los datos**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

**4. Descomprimir los datos**

In [ ]:
!unzip udea*.zip > /dev/null

In [ ]:
train = pd.read_csv("train.csv")
test  = pd.read_csv('test.csv')

# **ANÁLISIS EXPLORATORIO INICIAL**

Se muestran las primeras filas, tipos y valores nulos.

In [ ]:
if train is not None:
    display(train.head())
    display(train.info())
    display(train.isna().sum().sort_values(ascending=False).head(20))
else:
    print('No train available to display.')

# **PREPROCESAMIENTO**

In [ ]:
train_processed = train.copy()
inicio_total = time.time()

ID_COL = "ID"
TARGET = "RENDIMIENTO_GLOBAL"
NUM_VARS = ['PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
ESTRATO_MAP = {f'Estrato {i}': i for i in range(1, 7)}
REVERSE_ESTRATO = {v: k for k, v in ESTRATO_MAP.items()}
TEST_SIZE = 0.2
RANDOM_STATE = 42
SAMPLE_FRAC = 0.5

print(f"\nFilas originales: {len(train_processed):,}")

# SAMPLING
if len(train_processed) > 200000:
    print(f"Stratified sampling al {SAMPLE_FRAC*100:.0f}%...")
    train_sample, _ = train_test_split(
        train_processed,
        train_size=SAMPLE_FRAC,
        random_state=RANDOM_STATE,
        stratify=train_processed[TARGET]
    )
    train_processed = train_sample.reset_index(drop=True)
    print(f"Filas después: {len(train_processed):,}")

# LIMPIEZA
cols_drop = [c for c in train_processed.columns if ".1" in c]
train_processed.drop(columns=cols_drop, inplace=True, errors='ignore')

cat_vars = [c for c in train_processed.columns if c not in NUM_VARS + [ID_COL, TARGET]]
print(f"Numéricas: {len(NUM_VARS)} | Categóricas: {len(cat_vars)}")

# IMPUTACIÓN F_ESTRATOVIVIENDA
if "F_ESTRATOVIVIENDA" in train_processed.columns:
    socio_vars = [
        'F_TIENEAUTOMOVIL', 'F_TIENECOMPUTADOR', 'F_TIENELAVADORA',
        'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_EDUCACIONMADRE',
        'E_VALORMATRICULAUNIVERSIDAD'
    ]
    socio_vars = [v for v in socio_vars if v in train_processed.columns]

    temp = train_processed[socio_vars + ["F_ESTRATOVIVIENDA"]].copy()
    for col in socio_vars:
        temp[col] = temp[col].astype("category").cat.codes.replace({-1: np.nan})

    temp["F_ESTRATOVIVIENDA"] = temp["F_ESTRATOVIVIENDA"].map(ESTRATO_MAP)

    imputer = KNNImputer(n_neighbors=3, weights="distance")
    imputado = np.round(imputer.fit_transform(temp)[:, -1]).clip(1, 6)
    train_processed["F_ESTRATOVIVIENDA"] = pd.Series(imputado, index=train_processed.index).map(REVERSE_ESTRATO)

    del temp, imputer, imputado

# IMPUTACIÓN CATEGÓRICAS
for col in cat_vars:
    if train_processed[col].isnull().sum() > 0:
        fill_value = (
            train_processed[col].mode()[0]
            if train_processed[col].nunique() <= 3
            else "Desconocido"
        )
        train_processed[col] = train_processed[col].fillna(fill_value)

print("Imputación completada")

# OUTLIERS
for col in NUM_VARS:
    p1, p99 = train_processed[col].quantile([0.01, 0.99])
    train_processed[col] = train_processed[col].clip(p1, p99)

print("Outliers cappeados")

# TRAIN/TEST SPLIT
X = train_processed[NUM_VARS + cat_vars]
y = train_processed[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

del train_processed, X, y
print(f"Train/Test split: {len(X_train):,} / {len(X_test):,}")

# TRANSFORMACIONES
preprocessor = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), NUM_VARS),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True, max_categories=50), cat_vars),
    ],
    remainder="drop"
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

cat_ohe_cols = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_vars)
final_cols = NUM_VARS + list(cat_ohe_cols)

print(f"Features finales: {len(final_cols)}")

# CODIFICAR TARGET
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

print(f"Clases codificadas: {dict(zip(le.classes_, range(len(le.classes_))))}")

# BALANCEO
class_counts = np.bincount(y_train_encoded)
imbalance_ratio = class_counts.max() / class_counts.min()

if imbalance_ratio > 2.5:
    print(f"Aplicando SMOTE (ratio: {imbalance_ratio:.2f})...")
    smote = SMOTE(
        random_state=RANDOM_STATE,
        n_jobs=-1,
        k_neighbors=3,
        sampling_strategy='not majority'
    )
    X_train_balanced, y_train_balanced = smote.fit_resample(
        X_train_processed,
        y_train_encoded
    )
else:
    print(f"Balanceo aceptable (ratio: {imbalance_ratio:.2f})")
    X_train_balanced, y_train_balanced = X_train_processed, y_train_encoded

print("\nPREPROCESAMIENTO COMPLETADO\n")


# **ENTRENAMIENTO**

In [ ]:
print("\nMODELO: XGBoost Classifier")
print("=" * 70)

# ENTRENAR
print("\nEntrenando XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    eval_metric='mlogloss'
)

xgb_model.fit(X_train_balanced, y_train_balanced, verbose=False)
print("Entrenamiento completado.")

# VALIDACIÓN CRUZADA
print("\nValidación cruzada (3-fold)...")
cv_scores = cross_val_score(
    xgb_model,
    X_train_balanced,
    y_train_balanced,
    cv=3,
    scoring='f1_weighted'
)
print(f"CV F1-Weighted: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

# PREDICCIONES
print("\nGenerando predicciones en TEST...")
y_pred_xgb = xgb_model.predict(X_test_processed)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_processed)

# MÉTRICAS
acc_xgb = accuracy_score(y_test_encoded, y_pred_xgb)
f1_xgb = f1_score(y_test_encoded, y_pred_xgb, average='weighted')

print("\nRESULTADOS XGBoost:")
print(f"  Accuracy:  {acc_xgb:.4f}")
print(f"  F1-Score:  {f1_xgb:.4f}")

print("\nREPORTE DE CLASIFICACIÓN:")
print(classification_report(y_test_encoded, y_pred_xgb, target_names=le.classes_, digits=4))

# MATRIZ DE CONFUSIÓN
cm = confusion_matrix(y_test_encoded, y_pred_xgb)
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    ax=ax,
    xticklabels=le.classes_,
    yticklabels=le.classes_
)
ax.set_title('Matriz de Confusión - XGBoost')
ax.set_ylabel('Real')
ax.set_xlabel('Predicción')
plt.tight_layout()
plt.show()

# IMPORTANCIA DE FEATURES
feature_importance = pd.DataFrame({
    'Feature': final_cols,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTOP 15 FEATURES MÁS IMPORTANTES:")
print(feature_importance.head(15).to_string(index=False))

fig, ax = plt.subplots(figsize=(10, 6))
feature_importance.head(15).sort_values('Importance').plot(
    kind='barh',
    x='Feature',
    y='Importance',
    ax=ax,
    color='steelblue'
)
ax.set_title('Top 15 Features - XGBoost', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

# GUARDAR MODELO
print("\nGuardando modelo y preprocesadores...")
with open("xgboost_model.pkl", "wb") as f:
    pickle.dump(xgb_model, f)
with open("preprocessor_xgb.pkl", "wb") as f:
    pickle.dump(preprocessor, f)
with open("label_encoder_xgb.pkl", "wb") as f:
    pickle.dump(le, f)

print("Entrenamiento finalizado.")

# **PREDICCIÓN EN DATASET COMPLETO**

In [ ]:
print("\nCargando dataset original sin samplear...")
test_proc = test.copy()

print(f"Test original shape: {test_proc.shape}")

# PREPROCESAR TEST
cols_drop = [c for c in test_proc.columns if ".1" in c]
test_proc.drop(columns=cols_drop, inplace=True, errors='ignore')

if "F_ESTRATOVIVIENDA" in test_proc.columns:
    socio_vars = [v for v in socio_vars if v in test_proc.columns]
    temp = test_proc[socio_vars + ["F_ESTRATOVIVIENDA"]].copy()

    for col in socio_vars:
        temp[col] = temp[col].astype("category").cat.codes.replace({-1: np.nan})

    temp["F_ESTRATOVIVIENDA"] = temp["F_ESTRATOVIVIENDA"].map(ESTRATO_MAP)

    from sklearn.impute import KNNImputer
    imputer = KNNImputer(n_neighbors=3, weights="distance")
    imputado = np.round(imputer.fit_transform(temp)[:, -1]).clip(1, 6)

    test_proc["F_ESTRATOVIVIENDA"] = (
        pd.Series(imputado, index=test_proc.index).map(REVERSE_ESTRATO)
    )

    del temp, imputer, imputado

# IMPUTACIÓN VARIABLES CATEGÓRICAS
for col in cat_vars:
    if test_proc[col].isnull().sum() > 0:
        fill_value = (
            test_proc[col].mode()[0]
            if test_proc[col].nunique() <= 3
            else "Desconocido"
        )
        test_proc[col] = test_proc[col].fillna(fill_value)

# WINSORIZACIÓN / LIMITAR EXTREMOS
for col in NUM_VARS:
    p1, p99 = test_proc[col].quantile([0.01, 0.99])
    test_proc[col] = test_proc[col].clip(p1, p99)

print("Test preprocesado.")

# TRANSFORMAR
X_test_full = test_proc[NUM_VARS + cat_vars]
X_test_full_processed = preprocessor.transform(X_test_full)

print(f"Transformaciones aplicadas: {X_test_full_processed.shape}")

# PREDECIR
print(f"\nRealizando predicciones en {len(X_test_full):,} muestras...")
y_pred_encoded = xgb_model.predict(X_test_full_processed)
y_pred = le.inverse_transform(y_pred_encoded)

print("Predicciones completadas.")

# **CREAR SUBMISSION**

In [ ]:
submission = pd.DataFrame({
    'ID': test_proc[ID_COL].values,
    'RENDIMIENTO_GLOBAL': y_pred
})

print("\nDistribución de predicciones:")
for cls in le.classes_:
    count = (y_pred == cls).sum()
    pct = count / len(y_pred) * 100
    print(f"  {cls}: {count:,} ({pct:.1f}%)")

# GUARDAR ARCHIVO
print("\nGuardando submission_xgboost.csv...")
submission.to_csv('submission_xgboost.csv', index=False)

print("\n" + "="*70)
print("Archivo generado: submission_xgboost.csv")
print(f"Filas: {len(submission):,}")
print(f"Columnas: {list(submission.columns)}")
print("\n" + submission.head(10).to_string(index=False))

tiempo_total = time.time() - inicio_total
print(f"\nTiempo total: {tiempo_total/60:.2f} minutos")
print("="*70)

In [ ]:
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f submission_xgboost.csv -m "XGBoost"